In [39]:
import pandas as pd
import numpy as np
import time

In [52]:
test_board = load_board_for_solver('guardian-04-09-21.csv')
print(test_board)

[['.' '.' '.' '.' '.' '.' '.' '9' '.']
 ['.' '7' '.' '.' '5' '3' '.' '.' '6']
 ['2' '.' '.' '.' '.' '.' '3' '.' '5']
 ['.' '.' '4' '.' '3' '5' '.' '.' '7']
 ['.' '.' '1' '.' '6' '.' '.' '.' '.']
 ['.' '.' '7' '.' '1' '2' '.' '.' '8']
 ['6' '.' '.' '.' '.' '.' '7' '.' '1']
 ['.' '3' '.' '.' '7' '1' '.' '.' '2']
 ['.' '.' '.' '.' '.' '.' '.' '4' '.']]


In [4]:
def load_board_for_solver(path):
    board_df = pd.read_csv(path, header=None)
    board = board_df.to_numpy()
    assert board.shape[0] == 9 and board.shape[1] == 9, "Incorrect dimensions. Board must be 9 x 9"
    return board

In [5]:
def is_complete(board):
    for i in range(0,board.shape[0]):
        for j in range(0, board.shape[1]):
            if board[i][j] not in list(map(str, range(1,10))):
                return False
    return True

assert is_complete(test_board) == False
filled_board = np.full((9,9), '1')
assert is_complete(filled_board) == True

In [6]:
def convert_position_to_indices(position):
    row_index = ord(position[0]) - 65
    column_index = ord(position[1]) - 49 
    return row_index, column_index

row_index, column_index = convert_position_to_indices("A1")
assert row_index == 0 and column_index == 0
row_index, column_index = convert_position_to_indices("I9")
assert row_index == 8 and column_index == 8

In [7]:
def is_square_free(position, board):
    row_index, column_index = convert_position_to_indices(position)
    if board[row_index][column_index] == '.':
        return True
    return False

assert is_square_free('A1', test_board) == True
assert is_square_free('C1', test_board) == False

In [8]:
def is_row_free(position, digit, board):
    row_index, column_index = convert_position_to_indices(position)
    if digit in list(board[row_index, :]):
        return False
    return True

assert is_row_free('A1', '1', test_board) == True
assert is_row_free('A1', '9', test_board) == False

In [9]:
def is_column_free(position, digit, board):
    row_index, column_index = convert_position_to_indices(position)
    if digit in list(board[:, column_index]):
        return False
    return True

assert is_column_free('A1', '8', test_board) == True
assert is_column_free('A1', '2', test_board) == False

In [10]:
def is_box_free(position, digit, board):
    row_index, column_index = convert_position_to_indices(position)
    vertical_box = row_index // 3
    horizontal_box = column_index // 3
    if digit in list(board[vertical_box*3: vertical_box*3 +3, horizontal_box*3: horizontal_box*3 + 3].flatten()):
        return False
    return True

assert is_box_free('H5', '2', test_board) == True
assert is_box_free('D4', '3', test_board) == False

In [11]:
def insert_digit(position, digit, board):
    row_index, column_index = convert_position_to_indices(position)
    board[row_index][column_index] = digit 

insert_digit('A1', '1', test_board)
assert test_board[0][0] == '1'
insert_digit('A1', '.', test_board)
assert test_board[0][0] == '.'

In [12]:
def make_move(position, digit, board):
    if not is_square_free(position, board):
        return False
    if not is_row_free(position, digit, board):
        return False
    if not is_column_free(position, digit, board):
        return False
    if not is_box_free(position, digit, board):
        return False
    insert_digit(position, digit, board)
    return True

make_move('D7', '1', test_board)
assert test_board[3][6] == '1'
insert_digit('D7', '.', test_board)
assert test_board[3][6] == '.'
assert make_move('A8', '1', test_board) == False

In [13]:
def next_position(position):
    next_row = next_column = ''
    if position[1] == '9':
        next_row = chr(ord(position[0])+1)
        next_column = '1'
    else:
        next_row = position[0]
        next_column = chr(ord(position[1])+1)
    return next_row + next_column
        
assert next_position('A1') == 'A2'
assert next_position('A9') == 'B1'

In [15]:
def fill_square(position, board):
    if is_square_free(position, board):
        for digit in list(map(str, range(1,10))):
            if make_move(position, digit, board):
                if position == 'I9':
                    return True
                if fill_square(next_position(position), board):
                    return True
                insert_digit(position, '.', board)
    else:
        if position == 'I9':
            return True 
        if fill_square(next_position(position), board):
            return True
        return False
    return False

In [45]:
def solve_board(board):
    t_start = time.clock()
    if fill_square('A1', board):
        t_end = time.clock()
        print("Time to solve (ms): ", round((t_end - t_start) * 1000, 1))
        print(board)
    else:
        print("Cannot be solved")

In [48]:
board = load_board_for_solver('guardian-04-09-21.csv')
solve_board(board)

Time to solve (ms):  1399.3
[['8' '5' '3' '6' '2' '7' '1' '9' '4']
 ['1' '7' '9' '4' '5' '3' '2' '8' '6']
 ['2' '4' '6' '1' '8' '9' '3' '7' '5']
 ['9' '2' '4' '8' '3' '5' '6' '1' '7']
 ['3' '8' '1' '7' '6' '4' '5' '2' '9']
 ['5' '6' '7' '9' '1' '2' '4' '3' '8']
 ['6' '9' '2' '3' '4' '8' '7' '5' '1']
 ['4' '3' '8' '5' '7' '1' '9' '6' '2']
 ['7' '1' '5' '2' '9' '6' '8' '4' '3']]


In [49]:
board = load_board_for_solver('guardian-09-10-21.csv')
solve_board(board)

Time to solve (ms):  1347.9
[['1' '9' '8' '5' '7' '4' '3' '6' '2']
 ['6' '2' '4' '9' '3' '1' '8' '5' '7']
 ['7' '3' '5' '6' '8' '2' '9' '1' '4']
 ['8' '6' '3' '1' '2' '7' '5' '4' '9']
 ['2' '5' '7' '4' '9' '8' '6' '3' '1']
 ['4' '1' '9' '3' '5' '6' '2' '7' '8']
 ['9' '7' '6' '2' '4' '5' '1' '8' '3']
 ['3' '4' '1' '8' '6' '9' '7' '2' '5']
 ['5' '8' '2' '7' '1' '3' '4' '9' '6']]


In [47]:
board = load_board_for_solver('guardian-04-10-21.csv')
solve_board(board)

Time to solve (ms):  639.8
[['3' '5' '9' '6' '8' '2' '4' '1' '7']
 ['2' '8' '6' '4' '1' '7' '5' '9' '3']
 ['4' '1' '7' '5' '9' '3' '8' '6' '2']
 ['7' '6' '1' '3' '5' '9' '2' '4' '8']
 ['9' '2' '3' '8' '7' '4' '1' '5' '6']
 ['8' '4' '5' '1' '2' '6' '7' '3' '9']
 ['6' '3' '8' '2' '4' '1' '9' '7' '5']
 ['5' '7' '4' '9' '6' '8' '3' '2' '1']
 ['1' '9' '2' '7' '3' '5' '6' '8' '4']]


In [53]:
manual_entry_board = np.array([
 ['.', '.', '.', '.', '.', '.', '.', '9', '.'],
 ['.', '7', '.', '.', '5', '3', '.', '.', '6'],
 ['2', '.', '.', '.', '.', '.', '3', '.', '5'],
 ['.', '.', '4', '.', '3', '5', '.', '.', '7'],
 ['.', '.', '1', '.', '6', '.', '.', '.', '.'],
 ['.', '.', '7', '.', '1', '2', '.', '.', '8'],
 ['6', '.', '.', '.', '.', '.', '7', '.', '1'],
 ['.', '3', '.', '.', '7', '1', '.', '.', '2'],
 ['.', '.', '.', '.', '.', '.', '.', '4', '.']])

solve_board(manual_entry_board)

Time to solve (ms):  2751.7
[['8' '5' '3' '6' '2' '7' '1' '9' '4']
 ['1' '7' '9' '4' '5' '3' '2' '8' '6']
 ['2' '4' '6' '1' '8' '9' '3' '7' '5']
 ['9' '2' '4' '8' '3' '5' '6' '1' '7']
 ['3' '8' '1' '7' '6' '4' '5' '2' '9']
 ['5' '6' '7' '9' '1' '2' '4' '3' '8']
 ['6' '9' '2' '3' '4' '8' '7' '5' '1']
 ['4' '3' '8' '5' '7' '1' '9' '6' '2']
 ['7' '1' '5' '2' '9' '6' '8' '4' '3']]
